## Missing cases between slope calculation and whole part data and filtering all cases with Etco2

In [59]:
import pandas as pd

# Load the two sheets into separate DataFrames
sheet1 = pd.read_excel("Missing etco2 find.xlsx", sheet_name="slope etco2")
sheet2 = pd.read_excel("Missing etco2 find.xlsx", sheet_name="part etco2")

# Assuming the column to compare is named 'caseid' in both sheets
missing_cases = sheet2[~sheet2['caseid'].isin(sheet1['caseid'])]

# Display the missing cases
print("Cases present in 'part etco2' and missing in 'slope etco2':")
print(missing_cases)

# Display the total number of missing cases
total_missing = missing_cases.shape[0]
print(f"\nTotal number of missing cases: {total_missing}")

# Optionally, save the result to a new Excel file
missing_cases.to_excel("missing_cases.xlsx", index=False)


Cases present in 'part etco2' and missing in 'slope etco2':
      caseid
0      10192
21     16343
32     23067
38     25192
39     25205
...      ...
1526  175748
1528  175846
1529  175912
1538  177179
1540  177409

[370 rows x 1 columns]

Total number of missing cases: 370


"Missing cases" file has cases which were missed out of slope calculation

In [58]:
import pandas as pd

# Load the 'part etco2' sheet into a DataFrame
sheet2 = pd.read_excel("Missing etco2 find.xlsx", sheet_name="part etco2")

# Group by 'caseid' and check if each group has any non-empty 'etco2' value
caseid_with_etco2 = sheet2.groupby('caseid')['etco2'].transform(lambda x: x.notna().any())

# Filter the DataFrame to keep only the caseids that have at least one non-empty 'etco2' value
sheet2_cleaned = sheet2[caseid_with_etco2]

# Display the cleaned data
print("Cleaned 'part etco2' data (with at least one non-empty etco2 value per caseid):")
print(sheet2_cleaned)

# Optionally, save the cleaned data to a new Excel file
sheet2_cleaned.to_excel("part_etco2_cleaned.xlsx", index=False)



Cleaned 'part etco2' data (with at least one non-empty etco2 value per caseid):
       caseid  time     start       end      etco2
71      10192   -28  17:55:53  17:56:53        NaN
72      10192   -27  17:56:53  17:57:53        NaN
73      10192   -26  17:57:53  17:58:53        NaN
74      10192   -25  17:58:53  17:59:53        NaN
75      10192   -24  17:59:53  18:00:53        NaN
...       ...   ...       ...       ...        ...
63241  177413    -5  21:30:53  21:31:53  12.682617
63242  177413    -4  21:31:53  21:32:53  13.308594
63243  177413    -3  21:32:53  21:33:53  12.922656
63244  177413    -2  21:33:53  21:34:53  12.886719
63245  177413    -1  21:34:53  21:35:53  12.435547

[48918 rows x 5 columns]


- Step 1: Merging new Epineprine and all PART EtCO2 values
- Step 2: have demographoics sheet too with the merged file.
- Step 3: create a copy of the file and start working on visualizations and correlation


In [86]:
import pandas as pd

# Load the two sheets from the Excel file
file_path = 'epi_etco2_work.xlsx'

# Load 'epifirst' sheet which contains caseid and first epinephrine time
epifirst_df = pd.read_excel(file_path, sheet_name='epifirst')

# Load 'epimul' sheet which contains caseid and multiple epinephrine times
epimul_df = pd.read_excel(file_path, sheet_name='epimul')

# Perform the merge
# 'outer' merge ensures that caseids from both sheets are retained.
merged_df = pd.merge(epifirst_df, epimul_df, on='caseId', how='outer')

# Save the merged result to a new file or continue processing
output_path = 'merged_epi_etco2_work.xlsx'
merged_df.to_excel(output_path, index=False)

print(f"Merged data saved to {output_path}")


Merged data saved to merged_epi_etco2_work.xlsx


In [87]:
import pandas as pd

# Load the merged data from the previous step
merged_file_path = 'merged_epi_etco2_work.xlsx'
merged_df = pd.read_excel(merged_file_path)

# Replace values in epivasoalntm with epi1 if both are present
merged_df['epivasoalntm'] = merged_df.apply(
    lambda row: row['epi1'] if pd.notna(row['epi1']) else row['epivasoalntm'], axis=1
)

# Save the updated result to a new file
output_path = 'updated_merged_epi_etco2_work.xlsx'
merged_df.to_excel(output_path, index=False)

print(f"Updated data saved to {output_path}")

Updated data saved to updated_merged_epi_etco2_work.xlsx


In [109]:
import pandas as pd

# Load Epinephrine data from Excel
epi_df = pd.read_excel('updated_merged_epi_etco2_work.xlsx')

# Define a function to add ":00" to time strings in epi1 that only have HH:MM
def add_seconds_if_missing(time_str):
    if pd.isna(time_str):
        return time_str
    time_str = str(time_str)
    if len(time_str) == 5:  # If the format is HH:MM
        return time_str + ':00'
    return time_str

# Apply the function to the epi1 column
epi_df['epi1'] = epi_df['epi1'].apply(add_seconds_if_missing)

# Convert the updated epi1 times to datetime format with error handling
epi_df['epi1'] = pd.to_datetime(epi_df['epi1'], format='%H:%M:%S', errors='coerce').dt.time

# Save the updated dataframe back to an Excel file
epi_df.to_excel('updated_merged_epi_etco2_with_seconds_epi1.xlsx', index=False)

print("Updated epinephrine times for epi1 with seconds saved to 'updated_merged_epi_etco2_with_seconds_epi1.xlsx'.")


Updated epinephrine times for epi1 with seconds saved to 'updated_merged_epi_etco2_with_seconds_epi1.xlsx'.


## Now we merged all the epinephrine values

Step 1:

Filter records which has EtCO2 values.

 we will directly compare the epinephrine times with the start and end times in your ETCO₂ data without melting the epinephrine data. We'll iterate over the rows and match each epinephrine time to the correct ETCO₂ time interval.



Step 2: Convert Times to Datetime Format
Ensure that the time columns are in the datetime format for accurate comparisons.

In [113]:
import pandas as pd

# Load ETCO₂ data from Excel
etco2_df = pd.read_excel('part_etco2_work1.xlsx')

# Load Epinephrine data from Excel
epi_df = pd.read_excel('final_epi_input.xlsx')

# Convert the start and end times in ETCO₂ data to time-only format
etco2_df['start'] = pd.to_datetime(etco2_df['start']).dt.time
etco2_df['end'] = pd.to_datetime(etco2_df['end']).dt.time

# Convert epinephrine times to time-only format for each epi column
for col in epi_df.columns[1:]:  # Skip the first column (caseid or patientId)
    epi_df[col] = pd.to_datetime(epi_df[col], format='%H:%M:%S', errors='coerce').dt.time

# At this point, etco2_df and epi_df contain properly formatted time columns
print("ETCO₂ and Epinephrine data loaded and time columns converted.")

# Find the patient IDs present in Epinephrine data but missing in ETCO₂ data
uncommon_in_epi = set(epi_df['patientId']).difference(set(etco2_df['patientId']))

# Filter epi_df to only include the caseids that are missing in ETCO₂
missing_in_etco2_df = epi_df[epi_df['patientId'].isin(uncommon_in_epi)]

# Count the number of patient IDs missing in ETCO₂
missing_in_etco2_count = missing_in_etco2_df['patientId'].nunique()

# Save the result to an Excel file
missing_in_etco2_df.to_excel('missing_in_etco2.xlsx', index=False)


# Print the unique number of caseids in both DataFrames
etco2_unique_caseids = etco2_df['patientId'].nunique()
epi_unique_caseids = epi_df['patientId'].nunique()

print(f"Number of unique caseids in ETCO₂ data: {etco2_unique_caseids}")
print(f"Number of unique caseids in Epinephrine data: {epi_unique_caseids}")


# Find the common patient IDs
common_caseids = set(etco2_df['patientId']).intersection(set(epi_df['patientId']))

# Print the count of common patient IDs
common_caseids_count = len(common_caseids)

print(f"Number of common caseids in both ETCO₂ and Epinephrine data: {common_caseids_count}")

# Print the count
print(f"Count of patient IDs present in Epinephrine but missing in ETCO₂: {missing_in_etco2_count}")
print(f"Patient IDs saved to 'missing_in_etco2.xlsx'.")

C:\Users\raya06\AppData\Local\Temp\2\ipykernel_7000\2213206541.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  etco2_df['start'] = pd.to_datetime(etco2_df['start']).dt.time
C:\Users\raya06\AppData\Local\Temp\2\ipykernel_7000\2213206541.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  etco2_df['end'] = pd.to_datetime(etco2_df['end']).dt.time


ETCO₂ and Epinephrine data loaded and time columns converted.
Number of unique caseids in ETCO₂ data: 1542
Number of unique caseids in Epinephrine data: 2722
Number of common caseids in both ETCO₂ and Epinephrine data: 1499
Count of patient IDs present in Epinephrine but missing in ETCO₂: 1223
Patient IDs saved to 'missing_in_etco2.xlsx'.


Step 3: Match Epinephrine Times to ETCO₂ Intervals
We will iterate through the ETCO₂ dataframe and check if any of the epinephrine times fall within the start and end times for the same patient.

In [115]:
# Create a new column to store matched epinephrine times
etco2_df['epi_time'] = None

# Iterate over the ETCO₂ rows
for i, etco2_row in etco2_df.iterrows():
    patient_id = etco2_row['patientId']
    start_time = etco2_row['start']
    end_time = etco2_row['end']
    
    # Find the corresponding patient in the epinephrine dataframe
    epi_times = epi_df[epi_df['patientId'] == patient_id]
    
    # Check if epi_times is not empty
    if not epi_times.empty:
        # Check each epinephrine time to see if it falls within the start and end times
        for col in epi_df.columns[1:]:  # Skip the first column (patientId)
            epi_time = epi_times[col].values[0]  # Get the epi time for this column
            if pd.notna(epi_time) and start_time <= epi_time <= end_time:
                etco2_df.at[i, 'epi_time'] = epi_time
                break  # Stop after finding the first match

# Save the result to a CSV file
etco2_df.to_excel('matched_etco2_epinephrine.xlsx', index=False)

In [116]:

# Filter the rows where both etco2 values and at least one epi_time are present
filtered_df = etco2_df[etco2_df['epi_time'].notna()]

# Get the unique patient IDs with both etco2 values and at least one epinephrine value
patient_ids_with_etco2_and_epi = filtered_df['patientId'].unique()

# Get the count of unique patient IDs
unique_patient_count = filtered_df['patientId'].nunique()

# Save the filtered data to an Excel file
filtered_df.to_excel('filtered_etco2_with_epi.xlsx', index=False)

# Output the patient IDs and the unique count
print(f"Patient IDs with ETCO₂ values and at least one epinephrine value: {patient_ids_with_etco2_and_epi}")
print(f"Count of unique patient IDs: {unique_patient_count}")

Patient IDs with ETCO₂ values and at least one epinephrine value: [ 10192  10267  10279 ... 177271 177409 177413]
Count of unique patient IDs: 1447


Filtering files whoch have both etco2 and epi values

Third Assumption: In the has_etco2_with_epi_in_range function, we check whether the epi_time falls within the ETCO₂ time range or within 5 minutes before the start time

In [7]:
import pandas as pd
from datetime import timedelta

# Load the processed data from the Excel file
file_path = 'matched_etco2_epinephrine.xlsx'
df = pd.read_excel(file_path, dtype={'start': str, 'end': str, 'epi_time': str})

# Function to convert time strings to datetime.time
def convert_to_time(value):
    try:
        return pd.to_datetime(value, format='%H:%M:%S', errors='coerce').time()
    except:
        return None

# Apply conversion functions to the 'start', 'end', and 'epi_time' columns
df['start'] = df['start'].apply(convert_to_time)
df['end'] = df['end'].apply(convert_to_time)
df['epi_time'] = df['epi_time'].apply(convert_to_time)

# Function to convert time to seconds for easier comparison
def time_to_seconds(t):
    if pd.isnull(t):
        return None
    return t.hour * 3600 + t.minute * 60 + t.second

# Step 1: Filter records that have more than 5 non-null ETCO₂ values
def filter_non_null_etco2(group):
    non_null_etco2 = group[group['etco2'].notna()]
    if len(non_null_etco2) > 5:
        return True
    return False

# Apply the filter for groups with more than 5 non-null ETCO₂ values
valid_patient_ids_step1 = df.groupby('patientId').filter(filter_non_null_etco2)['patientId'].unique()

# Filter the data to include only patient IDs with more than 5 non-null ETCO₂ values
filtered_df_step1 = df[df['patientId'].isin(valid_patient_ids_step1)]

# Step 2: Further filter the data to check if epi_time exists and if there are 5 non-null ETCO₂ values after epi_time
def filter_etco2_after_epi_time(group):
    non_null_etco2 = group[group['etco2'].notna()]
    epi_time = group['epi_time'].dropna().unique()  # Find unique epi_time values

    if len(epi_time) == 0:  # If no epi_time, skip this group
        return False

    epi_time = epi_time[0]  # Take the first epi_time if multiple are found
    epi_seconds = time_to_seconds(epi_time)

    # Filter for non-null ETCO₂ values after epi_time
    etco2_after_epi = non_null_etco2[non_null_etco2['start'].apply(time_to_seconds) > epi_seconds]

    # Check if there are at least 5 non-null ETCO₂ values after epi_time
    if len(etco2_after_epi) >= 5:
        return True
    return False

# Step 3: Filter records within -3 minutes and +5 minutes of epi_time
def filter_etco2_within_custom_window(group):
    non_null_etco2 = group[group['etco2'].notna()]
    epi_time = group['epi_time'].dropna().unique()  # Find unique epi_time values

    if len(epi_time) == 0:  # If no epi_time, skip this group
        return False

    epi_time = epi_time[0]  # Take the first epi_time if multiple are found
    epi_seconds = time_to_seconds(epi_time)

    # Define the time window: -3 minutes (180 seconds) and +5 minutes (300 seconds)
    lower_bound = epi_seconds - 3 * 60
    upper_bound = epi_seconds + 5 * 60

    # Filter for non-null ETCO₂ values within the custom window of -3 to +5 minutes
    etco2_within_window = non_null_etco2[non_null_etco2['start'].apply(time_to_seconds).between(lower_bound, upper_bound)]

    # Check if there are more than 3 non-null ETCO₂ values within this time window
    if len(etco2_within_window) >= 2:
        return True
    return False

# Apply the third filter for groups with more than 3 non-null ETCO₂ values within -3 to +5 minutes of epi_time
valid_patient_ids_step3 = filtered_df_step1.groupby('patientId').filter(filter_etco2_within_custom_window)['patientId'].unique()

# Filter the original data to include only the valid patient IDs from all filters
filtered_df_final = df[df['patientId'].isin(valid_patient_ids_step3)]

# Get the number of unique patient IDs in the final filtered data
unique_patient_count = filtered_df_final['patientId'].nunique()

# Save the final filtered data to a new Excel file
output_path = 'filtered_etco2_with_epi_in_range_minus_3min_plus_5min_final.xlsx'
filtered_df_final.to_excel(output_path, index=False)

print(f"Filtered data saved to {output_path}")
print(f"Number of unique patient IDs in the filtered data: {unique_patient_count}")


Filtered data saved to filtered_etco2_with_epi_in_range_minus_3min_plus_5min_final.xlsx
Number of unique patient IDs in the filtered data: 705


In [8]:
import pandas as pd
from datetime import timedelta

# Load the processed data from the Excel file
file_path = 'matched_etco2_epinephrine.xlsx'
df = pd.read_excel(file_path, dtype={'start': str, 'end': str, 'epi_time': str})

# Function to convert time strings to datetime.time
def convert_to_time(value):
    try:
        return pd.to_datetime(value, format='%H:%M:%S', errors='coerce').time()
    except:
        return None

# Apply conversion functions to the 'start', 'end', and 'epi_time' columns
df['start'] = df['start'].apply(convert_to_time)
df['end'] = df['end'].apply(convert_to_time)
df['epi_time'] = df['epi_time'].apply(convert_to_time)

# Function to convert time to seconds for easier comparison
def time_to_seconds(t):
    if pd.isnull(t):
        return None
    return t.hour * 3600 + t.minute * 60 + t.second

# Step 1: Filter records that have more than 5 non-null ETCO₂ values
def filter_non_null_etco2(group):
    non_null_etco2 = group[group['etco2'].notna()]
    if len(non_null_etco2) > 5:
        return True
    return False

# Apply the filter for groups with more than 5 non-null ETCO₂ values
valid_patient_ids_step1 = df.groupby('patientId').filter(filter_non_null_etco2)['patientId'].unique()

# Filter the data to include only patient IDs with more than 5 non-null ETCO₂ values
filtered_df_step1 = df[df['patientId'].isin(valid_patient_ids_step1)]

# Step 2: Filter records within -3 minutes and +5 minutes of epi_time
def filter_etco2_within_custom_window(group):
    non_null_etco2 = group[group['etco2'].notna()]
    epi_times = group['epi_time'].dropna().unique()  # Find unique epi_time values

    if len(epi_times) == 0:  # If no epi_time, skip this group
        return False

    for epi_time in epi_times:  # Iterate over all epi_times
        epi_seconds = time_to_seconds(epi_time)

        # Define the time window: -3 minutes (180 seconds) and +5 minutes (300 seconds)
        lower_bound = epi_seconds - 3 * 60
        upper_bound = epi_seconds + 5 * 60

        # Filter for non-null ETCO₂ values within the custom window of -3 to +5 minutes
        etco2_within_window = non_null_etco2[non_null_etco2['start'].apply(time_to_seconds).between(lower_bound, upper_bound)]

        # If there are more than 3 non-null ETCO₂ values within this time window, include the patient
        if len(etco2_within_window) >= 3:
            return True

    # If none of the epi_times meet the condition, exclude the patient
    return False

# Apply the third filter for groups with more than 3 non-null ETCO₂ values within -3 to +5 minutes of any epi_time
valid_patient_ids_step3 = filtered_df_step1.groupby('patientId').filter(filter_etco2_within_custom_window)['patientId'].unique()

# Filter the original data to include only the valid patient IDs from all filters
filtered_df_final = df[df['patientId'].isin(valid_patient_ids_step3)]

# Get the number of unique patient IDs in the final filtered data
unique_patient_count = filtered_df_final['patientId'].nunique()

# Save the final filtered data to a new Excel file
output_path = 'filtered_etco2_with_epi_in_range_minus_3min_plus_5min_final_multiple_epitime.xlsx'
filtered_df_final.to_excel(output_path, index=False)

print(f"Filtered data saved to {output_path}")
print(f"Number of unique patient IDs in the filtered data: {unique_patient_count}")


Filtered data saved to filtered_etco2_with_epi_in_range_minus_3min_plus_5min_final_multiple_epitime.xlsx
Number of unique patient IDs in the filtered data: 660


Visualizing Etco2 values before combining new epinephrine values(Imputation)

## Random plots

In [26]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Load the filtered data from the main sheet
filtered_df = pd.read_excel('filtered_etco2_with_epi_in_range_or_5min_non_continuous.xlsx', sheet_name='Sheet1')

# Load the caseid and mnrosc data from the second sheet
mnrosc_df = pd.read_excel('filtered_etco2_with_epi_in_range_or_5min_non_continuous.xlsx', sheet_name='Sheet2')

# Convert start, end, and epi_time columns to time-only format
filtered_df['start'] = pd.to_datetime(filtered_df['start'], format='%H:%M:%S').dt.time
filtered_df['epi_time'] = pd.to_datetime(filtered_df['epi_time'], format='%H:%M:%S', errors='coerce').dt.time

# Specify the folder where the images will be saved
output_folder = 'ETCO2_Plots'

# Create the folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Randomly select 10 patient IDs from the dataset
top_patient_ids = filtered_df['patientId'].drop_duplicates().sample(n=10, random_state=42)

# Plot ETCO₂ waveforms for the randomly selected patients and save them
for patient_id in top_patient_ids:
    patient_data = filtered_df[filtered_df['patientId'] == patient_id].reset_index(drop=True)
    
    # Match patient_id with caseid in mnrosc_df
    mnrosc_value = mnrosc_df.loc[mnrosc_df['caseid'] == patient_id, 'mnrosc'].values
    if len(mnrosc_value) > 0:
        mnrosc_value = mnrosc_value[0]
    else:
        mnrosc_value = None
    
    # Determine the color based on mnrosc value
    if mnrosc_value == 0:
        line_color = 'blue'
        mnrosc_label = 'mnrosc = 0'
    elif mnrosc_value == 1:
        line_color = 'green'
        mnrosc_label = 'mnrosc = 1'
    else:
        line_color = 'gray'  # Default color if mnrosc is not found
        mnrosc_label = 'mnrosc not found'
    
    # Create a time index based on the row number
    time_index = patient_data.index
    
    # Plot the ETCO₂ values over the time index
    plt.figure(figsize=(12, 6))
    plt.plot(time_index, patient_data['etco2'], label=f'ETCO₂ Levels ({mnrosc_label})', color=line_color)
    
    # Mark the epinephrine time on the waveform
    epi_times = patient_data[patient_data['epi_time'].notna()].index
    for epi_time in epi_times:
        plt.axvline(x=epi_time, color='red', linestyle='--', label='Epinephrine Given')
    
    plt.title(f'ETCO₂ Levels for Patient {patient_id}')
    plt.xlabel('Time Index')
    plt.ylabel('ETCO₂ Levels')
    plt.legend()
    plt.grid(True)
    
    # Save the plot to the specified folder with the patient ID as the filename
    output_path = os.path.join(output_folder, f'Patient_{patient_id}.png')
    plt.savefig(output_path)
    
    # Close the plot to free memory
    plt.close()

print(f"Random 10 plots saved in the folder: {output_folder}")


Random 10 plots saved in the folder: ETCO2_Plots


## Top continuous plots

In [27]:
import os
import pandas as pd
import matplotlib.pyplot as plt


# Load the filtered data from the main sheet
filtered_df = pd.read_excel('filtered_etco2_with_epi_in_range_or_5min_non_continuous.xlsx', sheet_name='Sheet1')

# Load the caseid and mnrosc data from the second sheet
mnrosc_df = pd.read_excel('filtered_etco2_with_epi_in_range_or_5min_non_continuous.xlsx', sheet_name='Sheet2')

# Convert start, end, and epi_time columns to time-only format
filtered_df['start'] = pd.to_datetime(filtered_df['start'], format='%H:%M:%S').dt.time
filtered_df['epi_time'] = pd.to_datetime(filtered_df['epi_time'], format='%H:%M:%S', errors='coerce').dt.time

# Specify the folder where the images will be saved
output_folder = 'ETCO2_Plots_Top'

# Create the folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Plot ETCO₂ waveforms for the top patients and save them
for patient_id in top_patient_ids:
    patient_data = filtered_df[filtered_df['patientId'] == patient_id].reset_index(drop=True)
    
    # Match patient_id with caseid in mnrosc_df
    mnrosc_value = mnrosc_df.loc[mnrosc_df['caseid'] == patient_id, 'mnrosc'].values
    if len(mnrosc_value) > 0:
        mnrosc_value = mnrosc_value[0]
    else:
        mnrosc_value = None
    
    # Determine the color based on mnrosc value
    if mnrosc_value == 0:
        line_color = 'blue'
        mnrosc_label = 'mnrosc = 0'
    elif mnrosc_value == 1:
        line_color = 'green'
        mnrosc_label = 'mnrosc = 1'
    else:
        line_color = 'gray'  # Default color if mnrosc is not found
        mnrosc_label = 'mnrosc not found'
    
    # Create a time index based on the row number
    time_index = patient_data.index
    
    # Plot the ETCO₂ values over the time index
    plt.figure(figsize=(12, 6))
    plt.plot(time_index, patient_data['etco2'], label=f'ETCO₂ Levels ({mnrosc_label})', color=line_color)
    
    # Mark the epinephrine time on the waveform
    epi_times = patient_data[patient_data['epi_time'].notna()].index
    for epi_time in epi_times:
        plt.axvline(x=epi_time, color='red', linestyle='--', label='Epinephrine Given')
    
    plt.title(f'ETCO₂ Levels for Patient {patient_id}')
    plt.xlabel('Time Index')
    plt.ylabel('ETCO₂ Levels')
    plt.legend()
    plt.grid(True)
    
    # Save the plot to the specified folder with the patient ID as the filename
    output_path = os.path.join(output_folder, f'ETCO2_Patient_{patient_id}.png')
    plt.savefig(output_path)
    
    # Close the plot to free memory
    plt.close()

print(f"Plots saved in the folder: {output_folder}")


Plots saved in the folder: ETCO2_Plots_Top
